Mundo pequeño

# Búsqueda del vecino más cercano aproximado mediante grafos jerárquicos navegables de mundo pequeño
En esta libreta se realiza un buscador del vecino más cercano aproximado usando grafos jerárquicos navegables de mundo pequeño (Hnswlib). 

In [ ]:
from os import listdir
from os.path import isfile, join
import struct
import os 

import numpy as np

Instalamos la biblioteca [Hnswlib](https://github.com/nmslib/hnswlib)

Hierarchical Navigable Small Worlds (HNSW)

Nos ayuda para realizar una búsqueda rápida del vecino más cercano aproximado.



In [ ]:
!pip install hnswlib
import hnswlib

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for hnswlib: filename=hnswlib-0.6.2-cp37-cp37m-linux_x86_64.whl size=1450206 sha256=bf58799b204adde6c744c7a70a588714330aa3bacc543785272e0f37a14ca77c
  Stored in directory: /root/.cache/pip/wheels/67/01/80/9805daef8cd398ceb20003af220f77c4689cab8e43d466481b
Successfully built hnswlib


## Conjunto de datos
Para evaluar el buscador vamos usar el conjunto de vectores SIFT [ANN_SIFT10K](http://corpus-texmex.irisa.fr/) del grupo TEXMEX, el cual descargamos y extraemos.

In [ ]:
!wget -q ftp://ftp.irisa.fr/local/texmex/corpus/siftsmall.tar.gz
!tar xvzf siftsmall.tar.gz

siftsmall/
siftsmall/siftsmall_base.fvecs
siftsmall/siftsmall_groundtruth.ivecs
siftsmall/siftsmall_learn.fvecs
siftsmall/siftsmall_query.fvecs


Definimos una función para leer los vectores de un archivo `.fvecs`.

In [ ]:
import struct
import os 

def lee_fvecs(ruta):
  with open(ruta, 'rb') as f:
    d = struct.unpack('i', f.read(4))[0]
    n = f.seek(0, os.SEEK_END) // (4 + 4 * d)
    f.seek(0)
    vecs = np.zeros((n, d))
    for i in range(n):
      f.read(4)
      vecs[i] = struct.unpack('f' * d, f.read(d * 4))
  
  return vecs 

Leemos el conjunto de vectores base y consulta.

In [ ]:
base = lee_fvecs('siftsmall/siftsmall_base.fvecs')
consultas = lee_fvecs('siftsmall/siftsmall_query.fvecs')

print('Base: {0} Consultas: {1}'.format(base.shape, consultas.shape))

Base: (10000, 128) Consultas: (100, 128)


Definimos una función para leer los vectores más cercanos reales (_groundtruth_) de un archivo `.ivecs`

In [ ]:
def lee_ivecs(ruta):
  with open(ruta, 'rb') as f:
    d = struct.unpack('i', f.read(4))[0]
    n = f.seek(0, os.SEEK_END) // (4 + 4 * d)
    f.seek(0)
    vecs = np.zeros((n, d), dtype=np.int)
    for i in range(n):
      f.read(4)
      vecs[i] = struct.unpack('i' * d, f.read(d * 4))
  
  return vecs 

Leemos estos vectores.

In [ ]:
gt = lee_ivecs('siftsmall/siftsmall_groundtruth.ivecs')
print('Groundtruth: {0}'.format(gt.shape))

Groundtruth: (100, 100)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


## Distancia $\ell_2$
Creamos un índice de Hnswlib para la distancia $\ell_2$, configuramos los hiperparámetros y almacenamos el conjunto base.

In [ ]:
# Creates a non-initialized index an HNSW in space space with integer dimension dim.
p = hnswlib.Index(space='l2', dim=base.shape[1]) 
# Furthermore, Cosine similarity and Inner product can also be employed

# Initializes the index from with no elements.
p.init_index(max_elements=base.shape[0], ef_construction=100, M=16)
# max_elements defines the maximum number of elements that can be stored in the structure (can be increased).
# M defines tha maximum number of outgoing connections in the graph.
# ef_construction defines a construction time/accuracy trade-off.

# Controlling the recall by setting ef:
p.set_ef(10)
# Higher ef leads to better accuracy, but slower search

# Set the default number of cpu threads used during data insertion/querying, and batch search/construction
p.set_num_threads(4)
# By default using all available cores

# Element insertion. 
# Inserts the data(numpy array of vectors, shape:N*dim) into the structure.
p.add_items(base)

Realizamos las consultas usando este índice.

In [ ]:
# knn_query: make a batch query for k closest elements for each element of the
# data (shape:N*dim). 
# num_threads sets the number of cpu threads to use (-1 means use default).
# Thread-safe with other knn_query calls, but not with add_items.

nns_l2, l2_dists = p.knn_query(consultas, k=1)
# nns_l2 (Nearest-neighbors using L2 distance)
# l2_dists (Distances with the neighbors)
# Returns a numpy array of (shape:N*k).

Extraemos los vecinos más cercanos encontrados por Hswlib y los de referencia y los comparamos.

In [ ]:
# Ground truth vectors (vecinos más cercanos reales)
vmc_real = [g[0] for g in gt]
correcto = [nns_l2[i] == vmc_real[i] for i in range(len(nns_l2))]

# Comparación con los vectores encontrados por nns_l2 (Nearest-neighbors using L2 distance)
print('Promedio encontrados = {0}'.format(np.mean(correcto)))

Promedio encontrados = 0.95


## Ejercicio
 * Compara el desempeño de los algoritmos usando distintos hiperparámetros
 * Usa otro conjunto de datos para evaluar los algoritmos